# Solving Sudoku using Genetic Algorithm

In [1]:
import numpy as np
import random
import operator
from past.builtins import range

random.seed()

Nd = 9  

In [2]:
class Population(object):
    
    def __init__(self):
        self.candidates = []
        return

    def seed(self, Nc, given):
        self.candidates = []

        helper = Candidate()
        helper.values = [[[] for j in range(0, Nd)] for i in range(0, Nd)]
        for row in range(0, Nd):
            for column in range(0, Nd):
                for value in range(1, 10):
                    if ((given.values[row][column] == 0) and not (given.is_column_duplicate(column, value) or 
                                given.is_block_duplicate(row, column, value) or given.is_row_duplicate(row, value))):
                        helper.values[row][column].append(value)
                    elif given.values[row][column] != 0:
                        helper.values[row][column].append(given.values[row][column])
                        break

        for p in range(0, Nc):
            g = Candidate()
            for i in range(0, Nd):  
                row = np.zeros(Nd)
                for j in range(0, Nd):

                    if given.values[i][j] != 0:
                        row[j] = given.values[i][j]
                    elif given.values[i][j] == 0:
                        row[j] = helper.values[i][j][random.randint(0, len(helper.values[i][j]) - 1)]

                ii = 0
                while len(list(set(row))) != Nd:
                    ii += 1
                    if ii > 500000:
                        return 0
                    for j in range(0, Nd):
                        if given.values[i][j] == 0:
                            row[j] = helper.values[i][j][random.randint(0, len(helper.values[i][j]) - 1)]

                g.values[i] = row
            self.candidates.append(g)
        self.update_fitness()

        # print("Seeding complete.")

        return 1

    def update_fitness(self):
        for candidate in self.candidates:
            candidate.update_fitness()
        return

    def sort(self):
        self.candidates = sorted(self.candidates, key=operator.attrgetter('fitness'))
        return



In [8]:
class Candidate(object):

    def __init__(self):
        self.values = np.zeros((Nd, Nd))
        self.fitness = None
        return

    def update_fitness(self):        
        row_count = np.zeros(Nd)
        column_count = np.zeros(Nd)
        block_count = np.zeros(Nd)
        row_sum = 0
        column_sum = 0
        block_sum = 0

        self.values = self.values.astype(int) # avinash
        
        for j in range(0, Nd):
            for i in range(0, Nd):
                row_count[self.values[i][j] - 1] += 1

            row_sum += (1.0/len(set(row_count)))/Nd
            row_count = np.zeros(Nd)    

        for j in range(0, Nd):
            for i in range(0, Nd):
                column_count[self.values[i][j] - 1] += 1

            column_sum += (1.0/len(set(column_count)))/Nd
            column_count = np.zeros(Nd)    

        for i in range(0, Nd, 3):
            for j in range(0, Nd, 3):
                block_count[self.values[i][j] - 1] += 1
                block_count[self.values[i][j + 1] - 1] += 1
                block_count[self.values[i][j + 2] - 1] += 1

                block_count[self.values[i + 1][j] - 1] += 1
                block_count[self.values[i + 1][j + 1] - 1] += 1
                block_count[self.values[i + 1][j + 2] - 1] += 1

                block_count[self.values[i + 2][j] - 1] += 1
                block_count[self.values[i + 2][j + 1] - 1] += 1
                block_count[self.values[i + 2][j + 2] - 1] += 1

                block_sum += (1.0 / len(set(block_count))) / Nd
                block_count = np.zeros(Nd)

        if (int(column_sum) == 1 and int(block_sum) == 1 and int(row_sum) == 1 ):
            fitness = 1.0
        else:
            fitness = column_sum * block_sum

        self.fitness = fitness
        return

    def mutate(self, mutation_rate, given):
        
        r = random.uniform(0, 1.1)
        while r > 1: 
            r = random.uniform(0, 1.1)

        success = False
        if r < mutation_rate: 
            while not success:
                row1 = random.randint(0, 8)
                row2 = random.randint(0, 8)
                row2 = row1

                from_column = random.randint(0, 8)
                to_column = random.randint(0, 8)
                while from_column == to_column:
                    from_column = random.randint(0, 8)
                    to_column = random.randint(0, 8)

                if given.values[row1][from_column] == 0 and given.values[row1][to_column] == 0:
                    if (not given.is_column_duplicate(to_column, self.values[row1][from_column])) and (not given.is_column_duplicate(from_column, self.values[row2][to_column])) and (not given.is_block_duplicate(row2, to_column, self.values[row1][from_column])) and (not given.is_block_duplicate(row1, from_column, self.values[row2][to_column])):
                        
                        temp = self.values[row2][to_column]
                        self.values[row2][to_column] = self.values[row1][from_column]
                        self.values[row1][from_column] = temp
                        success = True

        return success




In [9]:
class Fixed(Candidate):

    def __init__(self, values):
        self.values = values
        return

    def is_row_duplicate(self, row, value):
        for column in range(0, Nd):
            if self.values[row][column] == value:
                return True
        return False

    def is_column_duplicate(self, column, value):
        for row in range(0, Nd):
            if self.values[row][column] == value:
                return True
        return False

    def is_block_duplicate(self, row, column, value):
        i = 3 * (int(row / 3))
        j = 3 * (int(column / 3))

        if ((self.values[i][j] == value) or (self.values[i][j + 1] == value) or 
            (self.values[i][j + 2] == value) or (self.values[i + 1][j] == value) or 
            (self.values[i + 1][j + 1] == value) or (self.values[i + 1][j + 2] == value) or 
            (self.values[i + 2][j] == value) or (self.values[i + 2][j + 1] == value) or 
            (self.values[i + 2][j + 2] == value)):
            return True
        else:
            return False

    def make_index(self, v):
        if v <= 2:
            return 0
        elif v <= 5:
            return 3
        else:
            return 6

    def no_duplicates(self):
        for row in range(0, Nd):
            for col in range(0, Nd):
                if self.values[row][col] != 0:
                    cnt1 = list(self.values[row]).count(self.values[row][col])
                    cnt2 = list(self.values[:,col]).count(self.values[row][col])

                    block_values = [y[self.make_index(col):self.make_index(col)+3] for y in
                                    self.values[self.make_index(row):self.make_index(row)+3]]
                    block_values_ = [int(x) for y in block_values for x in y]
                    cnt3 = block_values_.count(self.values[row][col])

                    if cnt1 > 1 or cnt2 > 1 or cnt3 > 1:
                        return False
        return True


In [10]:
class Tournament(object):
   
    def __init__(self):
        return

    def compete(self, candidates):
        c1 = candidates[random.randint(0, len(candidates) - 1)]
        c2 = candidates[random.randint(0, len(candidates) - 1)]
        f1 = c1.fitness
        f2 = c2.fitness

        if (f1 > f2):
            fittest = c1
            weakest = c2
        else:
            fittest = c2
            weakest = c1

        selection_rate = 0.80
        r = random.uniform(0, 1.1)
        while (r > 1): 
            r = random.uniform(0, 1.1)
        
        if (r < selection_rate):
            return fittest
        else:
            return weakest


In [11]:
class CycleCrossover(object):
    
    def __init__(self):
        return

    def crossover(self, parent1, parent2, crossover_rate):
        child1 = Candidate()
        child2 = Candidate()

        child1.values = np.copy(parent1.values)
        child2.values = np.copy(parent2.values)

        r = random.uniform(0, 1.1)
        while (r > 1):  
            r = random.uniform(0, 1.1)

        if (r < crossover_rate):
            crossover_point1 = random.randint(0, 8)
            crossover_point2 = random.randint(1, 9)
            while (crossover_point1 == crossover_point2):
                crossover_point1 = random.randint(0, 8)
                crossover_point2 = random.randint(1, 9)

            if (crossover_point1 > crossover_point2):
                temp = crossover_point1
                crossover_point1 = crossover_point2
                crossover_point2 = temp

            for i in range(crossover_point1, crossover_point2):
                child1.values[i], child2.values[i] = self.crossover_rows(child1.values[i], child2.values[i])

        return child1, child2

    def crossover_rows(self, row1, row2):
        child_row1 = np.zeros(Nd)
        child_row2 = np.zeros(Nd)

        remaining = range(1, Nd + 1)
        cycle = 0

        while ((0 in child_row1) and (0 in child_row2)):  
            
            if (cycle % 2 == 0):  
                index = self.find_unused(row1, remaining)
                start = row1[index]
                remaining.remove(row1[index])
                child_row1[index] = row1[index]
                child_row2[index] = row2[index]
                next = row2[index]

                while (next != start):
                    index = self.find_value(row1, next)
                    child_row1[index] = row1[index]
                    remaining.remove(row1[index])
                    child_row2[index] = row2[index]
                    next = row2[index]

                cycle += 1

            else: 
                index = self.find_unused(row1, remaining)
                start = row1[index]
                remaining.remove(row1[index])
                child_row1[index] = row2[index]
                child_row2[index] = row1[index]
                next = row2[index]

                while (next != start): 
                    index = self.find_value(row1, next)
                    child_row1[index] = row2[index]
                    remaining.remove(row1[index])
                    child_row2[index] = row1[index]
                    next = row2[index]

                cycle += 1

        return child_row1, child_row2

    def find_unused(self, parent_row, remaining):
        for i in range(0, len(parent_row)):
            if (parent_row[i] in remaining):
                return i

    def find_value(self, parent_row, value):
        for i in range(0, len(parent_row)):
            if (parent_row[i] == value):
                return i


In [12]:
class Sudoku(object):
    
    def __init__(self):
        self.given = None
        return
    
    def load(self, p):
        self.given = Fixed(p)
        return
    
    def save(self, path, solution):
        with open(path, "w") as f:
            np.savetxt(f, solution.values.reshape(Nd*Nd), fmt='%d')
        return
        
    def solve(self):

        Nc = 1000  
        Ne = int(0.05 * Nc) 
        Ng = 10000
        Nm = 0  

        phi = 0
        sigma = 1
        mutation_rate = 0.06

        if self.given.no_duplicates() == False:
            return (-1, 1)

        self.population = Population()
        #print("Created an initial population !!")

        if self.population.seed(Nc, self.given) ==  1:
            pass
        else:
            return (-1, 1)

        stale = 0
        for generation in range(0, Ng):

            best_fitness = 0.0
            for c in range(0, Nc):
                fitness = self.population.candidates[c].fitness
                if (fitness == 1):
                    #print("Solution found at generation %d!" % generation)
                    return (generation, self.population.candidates[c])

                if (fitness > best_fitness):
                    best_fitness = fitness
                
            #print("Generation:", generation, " Best fitness:", best_fitness)
            
            next_population = []

            self.population.sort()
            elites = []
            for e in range(0, Ne):
                elite = Candidate()
                elite.values = np.copy(self.population.candidates[e].values)
                elites.append(elite)

            for count in range(Ne, Nc, 2):
                t = Tournament()
                parent1 = t.compete(self.population.candidates)
                parent2 = t.compete(self.population.candidates)

                cc = CycleCrossover()
                child1, child2 = cc.crossover(parent1, parent2, crossover_rate=1.0)

                child1.update_fitness()
                old_fitness = child1.fitness
                success = child1.mutate(mutation_rate, self.given)
                child1.update_fitness()
                if (success):
                    Nm += 1
                    if (child1.fitness > old_fitness): 
                        phi = phi + 1

                child2.update_fitness()
                old_fitness = child2.fitness
                success = child2.mutate(mutation_rate, self.given)
                child2.update_fitness()
                if (success):
                    Nm += 1
                    if (child2.fitness > old_fitness):  
                        phi = phi + 1

                next_population.append(child1)
                next_population.append(child2)

            for e in range(0, Ne):
                next_population.append(elites[e])

            self.population.candidates = next_population
            self.population.update_fitness()

            if (Nm == 0):
                phi = 0  
            else:
                phi = phi / Nm

            if (phi > 0.2):
                sigma = sigma / 0.998
            elif (phi < 0.2):
                sigma = sigma * 0.998

            mutation_rate = abs(np.random.normal(loc=0.0, scale=sigma, size=None))

            self.population.sort()
            if (self.population.candidates[0].fitness != self.population.candidates[1].fitness):
                stale = 0
            else:
                stale += 1

            if (stale >= 100):
                #print("The population has gone stale. Re-seeding...")
                self.population.seed(Nc, self.given)
                stale = 0
                sigma = 1
                phi = 0
                mutation_rate = 0.06

        print("No solution found.")
        return (-2, 1)

In [13]:
class Solution_Sudoku(object):
    def __init__(self):
        
        self.given = "080000090007502800600807005370080051200000008950040032800104009001903600040000020"
        self.grid = np.array(list(self.given)).reshape((9,9)).astype(int)
        print(self.grid)
    
    def print_solution_matrics(self,path):
        with open(path, "r") as f:
            values = np.loadtxt(f).reshape((9, 9)).astype(int)
        print(values)
        
    def solve_it(self):
        s = Sudoku()
        s.load(self.grid)
        generation, solution = s.solve()
        if(solution):
            #print(type(solution))
            s.save("solution1.txt",solution)
            self.print_solution_matrics("solution1.txt")

In [14]:
sol = Solution_Sudoku()
sol.solve_it()

[[0 8 0 0 0 0 0 9 0]
 [0 0 7 5 0 2 8 0 0]
 [6 0 0 8 0 7 0 0 5]
 [3 7 0 0 8 0 0 5 1]
 [2 0 0 0 0 0 0 0 8]
 [9 5 0 0 4 0 0 3 2]
 [8 0 0 1 0 4 0 0 9]
 [0 0 1 9 0 3 6 0 0]
 [0 4 0 0 0 0 0 2 0]]
[[1 8 5 4 3 6 2 9 7]
 [4 3 7 5 9 2 8 1 6]
 [6 9 2 8 1 7 3 4 5]
 [3 7 6 2 8 9 4 5 1]
 [2 1 4 3 7 5 9 6 8]
 [9 5 8 6 4 1 7 3 2]
 [8 6 3 1 2 4 5 7 9]
 [7 2 1 9 5 3 6 8 4]
 [5 4 9 7 6 8 1 2 3]]
